In [20]:
##############################
#    Generating histogram    #
##############################

""""""

''

From this point all the images and annotations that were previously in the train and valid folder, were deleted; and images genereted into the cells folder, along the annotations.txt file were moved outside the cells folder to the train and valid folders. 

In [1]:
import os
from PIL import Image, ImageDraw

# Auxiliary Functions

# Draw a circle on an image.
def draw_circle_on_image(img, center, edge=None, fixed_radius=None):
    """Draw a circle on the given image based on center and edge or a fixed radius."""
    draw = ImageDraw.Draw(img)
    if not edge:
        bbox = [center[0]-fixed_radius, center[1]-fixed_radius, center[0]+fixed_radius, center[1]+fixed_radius]
    else:
        radius = int(((center[0]-edge[0])**2 + (center[1]-edge[1])**2)**0.5)
        bbox = [center[0]-radius, center[1]-radius, center[0]+radius, center[1]+radius]
    draw.ellipse(bbox, outline="green", width=2)

# Create a cropped image of individual figure.
def create_individual_figure_image(original_image, center=None, edge=None, fixed_radius=None, coordinates=None, margin=10):
    """Crop the image around the region of interest."""
    if coordinates:
        x_coords, y_coords = zip(*coordinates)
        left = max(min(x_coords) - margin, 0)
        top = max(min(y_coords) - margin, 0)
        right = min(max(x_coords) + margin, original_image.width)
        bottom = min(max(y_coords) + margin, original_image.height)
    else:
        if edge:
            radius = int(((center[0]-edge[0])**2 + (center[1]-edge[1])**2)**0.5)
        elif fixed_radius:
            radius = fixed_radius
        else:
            raise ValueError("Either edge or fixed_radius must be provided.")
        left = max(center[0] - radius - margin, 0)
        top = max(center[1] - radius - margin, 0)
        right = min(center[0] + radius + margin, original_image.width)
        bottom = min(center[1] + radius + margin, original_image.height)

    cropped_image = original_image.crop((left, top, right, bottom))
    return cropped_image

# Create a dictionary of annotation and photo file pairs.
def create_file_dict(folder_path):
    """Create a dictionary with annotation-photo file pairs."""
    files = os.listdir(folder_path)
    annotation_files = sorted([f for f in files if f.startswith("A") and f.endswith(".txt")])
    photo_files = sorted([f for f in files if f.startswith("P") and f.endswith(".jpg")])

    file_dict = {}
    for a in annotation_files:
        corresponding_photo = "P" + a[1:-3] + "jpg"
        if corresponding_photo in photo_files:
            file_dict[a] = corresponding_photo

    print(f"Total annotations found: {len(annotation_files)}")
    print(f"Total photos found: {len(photo_files)}")
    print(f"Total valid pairs found: {len(file_dict)}")
    return file_dict

# Ensure the 'cells' directory exists. If not, create it.
def ensure_directory_exists(folder_path, subfolder_name="cells_valid"):
    """Ensure a subfolder exists within a folder, create it if not."""
    directory_path = os.path.join(folder_path, subfolder_name)
    if not os.path.exists(directory_path):
        os.mkdir(directory_path)
    return directory_path

# Process images and annotations, create crops, and write information to a file.
def process_images_and_annotations(folder_path, file_dict):
    """Process image-annotation pairs, create crops and write crop info to a file."""
    n = 0  # Counter to number the crops

    # Ensure the cells directory exists and get its path
    cells_directory = ensure_directory_exists(folder_path)

    # File path for the output txt
    output_txt_path = os.path.join(cells_directory, "info_valid.txt")
   
    with open(output_txt_path, "w") as output_txt:  # Open file for writing
        for aid, pid in file_dict.items():
            annotation_path = os.path.join(folder_path, aid)
            photo_path = os.path.join(folder_path, pid)

            img = Image.open(photo_path)

            with open(annotation_path, 'r') as f:
                lines = f.readlines()[1:]
                for line in lines:
                    data = line.strip().split(',')
                    species = pid[1].upper()
                    if species == "F":
                        species_name = "falciparum"
                    elif species == "V":
                        species_name = "vivax"
                    else:
                        species_name = "uninfected"

                    cell_type = data[1]
                    comment = data[2]

                    center = (int(float(data[5])), int(float(data[6])))
                    if data[3] == "Circle":
                        edge = (int(float(data[7])), int(float(data[8])))
                        cropped_image = create_individual_figure_image(img, center, edge)
                    elif data[3] == "Polygon":
                        num_coords = int(data[4])
                        coordinates = [(float(data[5+2*i]), float(data[6+2*i])) for i in range(num_coords)]
                        cropped_image = create_individual_figure_image(img, coordinates=coordinates)
                    elif data[3] == "Point":
                        cropped_image = create_individual_figure_image(img, center, fixed_radius=60)

                    n += 1
                    cropped_image_path = os.path.join(cells_directory, f"{n}.jpg")
                    cropped_image.save(cropped_image_path)
                    output_txt.write(f"{n},{species_name},{cell_type.lower()},{comment.lower()}\n")

# Main entry point for the script.
if __name__ == "__main__":
    train_folder = "C:/Malaria/Dataset/valid"
    file_dict = create_file_dict(train_folder)
    process_images_and_annotations(train_folder, file_dict)


Total annotations found: 1147
Total photos found: 1147
Total valid pairs found: 1147
